In [8]:
import sys
sys.path.append('/MS3D')
import pickle
from pathlib import Path
from pcdet.utils.transform_utils import ego_to_world, world_to_ego
import numpy as np
np.set_printoptions(precision=4, suppress=True)
from pcdet.utils import generate_ps_utils
from visualize_bev import plot_boxes 
from pcdet.utils.tracker_utils import get_frame_track_boxes
from pcdet.utils import compatibility_utils as compat
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils
from pcdet.datasets import build_dataloader
def load_pkl(pth):
    with open(pth, 'rb') as f:
        pkl_file = pickle.load(f)
    return pkl_file

def plot_pointcloud(pts, limit_range, size=0.5, c='black'):
    plt.figure(figsize=(8,8))
    ax = plt.subplot(111)    
    mask = mask_points_by_range(pts, limit_range)
    ax.scatter(pts[mask][:,0],pts[mask][:,1],s=size, c=c, marker='o')
    return plt, ax


# cfg_file = 'cfgs/source-waymo/secondiou_ft_once.yaml'
# cfg_file = 'cfgs/target-nuscenes/ft_waymo_secondiou.yaml'
cfg_file = 'cfgs/target-waymo/ft_nuscenes_secondiou.yaml'
# cfg_file = 'cfgs/target-lyft/nuscenes_secondiou.yaml'
cfg_from_yaml_file(cfg_file, cfg)
cfg.DATA_CONFIG_TAR .DATA_SPLIT.test='train'
if cfg.DATA_CONFIG_TAR.get('USE_TTA', False):
    cfg.DATA_CONFIG_TAR.USE_TTA=False
    
cfg.DATA_CONFIG_TAR.USE_PSEUDO_LABEL=False
cfg.DATA_CONFIG_TAR.SAMPLED_INTERVAL.test=1
logger = common_utils.create_logger('temp.txt', rank=cfg.LOCAL_RANK)
target_set, target_loader, sampler = build_dataloader(
            dataset_cfg=cfg.DATA_CONFIG_TAR,
            class_names=cfg.DATA_CONFIG_TAR.CLASS_NAMES,
            batch_size=1, logger=logger, training=False, dist=False, workers=1
        )
idx_to_frameid = {v: k for k, v in target_set.frameid_to_idx.items()}

2023-04-17 03:37:36,791   INFO  Loading Waymo dataset
2023-04-17 03:37:36,791   INFO  Loading Waymo dataset
2023-04-17 03:37:36,791   INFO  Loading Waymo dataset
2023-04-17 03:37:36,791   INFO  Loading Waymo dataset
2023-04-17 03:37:38,988   INFO  Total skipped info 0
2023-04-17 03:37:38,988   INFO  Total skipped info 0
2023-04-17 03:37:38,988   INFO  Total skipped info 0
2023-04-17 03:37:38,988   INFO  Total skipped info 0
2023-04-17 03:37:38,990   INFO  Total samples for Waymo dataset: 37680
2023-04-17 03:37:38,990   INFO  Total samples for Waymo dataset: 37680
2023-04-17 03:37:38,990   INFO  Total samples for Waymo dataset: 37680
2023-04-17 03:37:38,990   INFO  Total samples for Waymo dataset: 37680


In [206]:
from pcdet.datasets.augmentor.augmentor_utils import get_points_in_box

box_list = []
accum_pts = []
seq_id = 'segment-9142545919543484617_86_000_106_000_with_camera_labels'
seq_infos = target_set.seq_name_to_infos[seq_id]
for seq_info in seq_infos:
    pose = compat.get_pose(target_set, seq_info['frame_id'])
    pts = compat.get_lidar(target_set, seq_info['frame_id'])
    pts, _ = ego_to_world(pose, points=pts)
    
    frame_boxes = seq_info['annos']['gt_boxes_lidar']
    _, frame_boxes = ego_to_world(pose, boxes=frame_boxes)
#     res = np.array([100,16612]) - frame_boxes[:,:2] # 13506499849906169066_120_000_140_000_with_camera_labels
    res = np.array([-525,522]) - frame_boxes[:,:2]
    mask = (abs(res[:,0]) < 1) & (abs(res[:,1]) < 1)
    mygt = frame_boxes[mask]
    if len(mygt) > 1:
        print('more than 1')
        break
    if len(mygt) == 0:
        continue
    mygt = mygt.squeeze()
    box_list.append(mygt)
    
    ground_lift = 0.12
    mygt += np.array([0,0,+ground_lift,0,0,-2*ground_lift,0,0,0])
    obj_pts, mask = get_points_in_box(pts, mygt)
    accum_pts.append(obj_pts)    
    

In [216]:
from visual_utils.open3d_vis_utils import translate_boxes_to_open3d_instance

all_gt_boxes = []
for box in box_list:    
    line_set, _ = translate_boxes_to_open3d_instance(box.squeeze(), 
                                                    use_linemesh=True, 
                                                     line_thickness=0.0007)
    for lines in line_set:
        lines.paint_uniform_color([0.2,0.4,0.6])
    all_gt_boxes.extend(line_set)

In [217]:
import open3d as o3d
from visual_utils.common_vis import convert_to_o3dpcd, convert_to_o3dbox

from matplotlib.cm import get_cmap
cmap = get_cmap('Paired')
# cmap = get_cmap('Set1')
colors = np.array(cmap.colors)[1:]
# colors = np.array([[0.651 , 0.8078, 0.8902],[0.8,0.4,0.2],[0.4,1,0.2],[0,0,1],[1,1,1]])
c_counter = 0
all_pcds = []
for enum, a_pts in enumerate(accum_pts):

    pcd = convert_to_o3dpcd(a_pts)
    pcd.paint_uniform_color(colors[c_counter % 20])
    all_pcds.append(pcd)
    
    if enum % 35 == 0:
        c_counter += 1
        
# o3dbox = convert_to_o3dbox(np.vstack(box_list), colour=[0.2,0.4,0.6])
o3d.visualization.draw_geometries(all_pcds + all_gt_boxes)
# o3d.visualization.draw_geometries(all_pcds)

In [12]:
frame_id = 'segment-9142545919543484617_86_000_106_000_with_camera_labels_102'
# centre at -555,517

pose = compat.get_pose(target_set, frame_id)
class_mask = np.array([n in target_set.class_names for n in compat.get_gt_names(target_set, frame_id)], 
                      dtype=np.bool_)
gt_boxes = compat.get_gt_boxes(target_set, frame_id)[class_mask]
_, gt_boxes = ego_to_world(pose, boxes=gt_boxes)

In [42]:
res = np.array([-555,517]) - gt_boxes[:,:2]
mask = (abs(res[:,0]) < 0.5) & (abs(res[:,1]) < 0.5)
gt_boxes[mask]

array([[-554.8728,  516.7043,   38.4496,    4.2482,    1.9889,    1.34  ,
           1.485 ,    0.0351,   -0.0092]])